In [14]:
import pandas as pd
import numpy as np
import openpyxl

In [15]:
raw_data = pd.read_csv("manaus.csv", sep=";", low_memory=False)
raw_data.head()

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,...,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
0,25/01/2020 00:00,03/01/2020 00:00,01/01/2020 00:00,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,TARUMA,TARUMÃ,OESTE,"0,243042897"
1,03/01/2020 00:00,03/01/2020 00:00,01/01/2020 00:00,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,...,Sim,Sim,Influenza B,NaN,NaN,NaN,TARUMA-ACU,TARUMÃ-AÇÚ,OESTE,"0,684931507"
2,05/01/2020 00:00,03/01/2020 00:00,03/01/2020 00:00,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,JORGE TEIXEIRA,JORGE TEIXEIRA,LESTE,"0,073137374"
3,16/01/2020 00:00,08/01/2020 00:00,04/01/2020 00:00,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,FLORES,FLORES,SUL,"0,198306463"
4,11/01/2020 00:00,08/01/2020 00:00,05/01/2020 00:00,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,CIDADE DE DEUS,CIDADE DE DEUS,NORTE,"0,117699678"


In [16]:
df_confirmed = raw_data[raw_data['_classificacao'] == 'Confirmado']
df_confirmed.head()

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,...,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
130,02/03/2020 00:00,23/02/2020 00:00,15/02/2020 00:00,Confirmado,Recuperado,"SIVEP, GAL",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,NaN,NaN,1.0,MAUAZINHO,MAUAZINHO,LESTE,"0,35040998"
144,01/03/2020 00:00,28/02/2020 00:00,26/02/2020 00:00,Confirmado,Recuperado,"SIVEP, GAL",NaN,NaN,NaN,NaN,...,Sim,Sim,Vírus Sincicial Respiratório,NaN,NaN,1.0,NOVO ALEIXO,NOVO ALEIXO,NORTE,"0,085452557"
153,05/04/2020 00:00,02/03/2020 00:00,23/02/2020 00:00,Confirmado,Recuperado,"SIVEP, eSUS VE,",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,1.0,NaN,NaN,PRACA 14 DE JANEIRO,PRAÇA 14 DE JANEIRO,SUL,"0,861920359"
158,14/03/2020 00:00,02/03/2020 00:00,29/02/2020 00:00,Confirmado,Recuperado,"SIVEP, eSUS VE,",Sim,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,2.0,NaN,NaN,FLORES,FLORES,SUL,"0,198306463"
196,14/03/2020 00:00,11/03/2020 00:00,09/03/2020 00:00,Confirmado,Recuperado,"SIVEP, GAL",NaN,NaN,NaN,NaN,...,Sim,Sim,SRAG não especificado,NaN,NaN,2.0,CIDADE NOVA,CIDADE NOVA,NORTE,"0,068152389"


## 2.1-1.Quantos atributos descrevem cada exemplo? Quais são eles?

In [17]:
quantity_attributes = df_confirmed.shape[1]
print(f'Dataframe possui {quantity_attributes} atributos.')

Dataframe possui 39 atributos.


In [18]:
attributes = pd.DataFrame(df_confirmed.dtypes, columns=['Tipo de dados'])
attributes.columns.name = 'Atributos'
attributes

Atributos,Tipo de dados
_dt_evolucao,object
_dt_notificacao,object
_dt_sintomas,object
_classificacao,object
_conclusao,object
_origem,object
_comorb_cardio,object
_comorb_cromossomica,object
_comorb_diabetes,object
_comorb_hemato,object


## 2.1-2.Quantos casos confirmados há em Manaus, cumulativamente?

In [19]:
confirmed_cases = df_confirmed.shape[0]
print(f'{confirmed_cases} casos confirmados em Manaus.')

80420 casos confirmados em Manaus.


## 2.1-3.A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.

In [46]:
df_confirmed_notification = pd.DataFrame(df_confirmed['_dt_notificacao'])
df_confirmed_notification['_dt_notificacao'] = pd.to_datetime(df_confirmed_notification['_dt_notificacao'], format='%d/%m/%Y %H:%M')

In [38]:
df_confirmed_notification.sort_values('_dt_notificacao', ascending=False, inplace=True)
df_confirmed_notification = df_confirmed_notification.dropna()

In [39]:
oldest_record = df_confirmed_notification.iloc[0]
latest_record = df_confirmed_notification.iloc[df_confirmed_notification.shape[0]-1]

In [45]:
print(f'Registro mais antigo é {oldest_record.values}')
print(f'Registro mais recente é {latest_record.values}')

Registro mais antigo é ['2020-12-29T00:00:00.000000000']
Registro mais recente é ['2020-02-16T00:00:00.000000000']
